In [1]:
import pandas_datareader.data as web
import pandas as pd
import pandas_ta
import numpy as np  
import yfinance as yf



# sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
# sp500['Symbol'] = sp500['Symbol'].str.replace('.','-')
# symbols_list = sp500['Symbol'].unique().tolist()
# to_remove = ['SOLV', 'VLTO', 'SW', 'GEV'] # Yfinance does not have legacy data for these tickers, consider reintegrating
# symbols_list = [symbol for symbol in symbols_list if symbol not in to_remove]

end_date = '2025-05-01'
start_date = pd.to_datetime(end_date) - pd.DateOffset(years=8)


df = yf.download(tickers = 'SPY',
                start = start_date,
                end = end_date,
                timeout= 5.0)
                # .stack(future_stack=True)
df.columns = df.columns.get_level_values(0) 
# print(df.columns.tolist())

df.columns = df.columns.str.lower()

df['garman_klass_vol'] = ((np.log(df['high'])-np.log(df['low']))**2)/2-(2*np.log(2)-1)*((np.log(df['close'])-np.log(df['open']))**2)

df['rsi'] = pandas_ta.rsi(df['close'], length=20)

# Chest RSI for AAPL to see if it works
# df.xs('SPY', level=1)['rsi'].plot(title='SPY RSI', figsize=(10, 5), color='blue') 

df['bb_low'] = df['close'].transform(lambda x:pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:, 0])
df['bb_mid'] = df['close'].transform(lambda x:pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:, 1])
df['bb_high'] = df['close'].transform(lambda x:pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:, 2])


def compute_atr(stock_data):
    atr = pandas_ta.atr(high=stock_data['high'],
        low=stock_data['low'],
        close=stock_data['close'],
        length=14)
    atr.dropna(inplace=True)
    z_atr = atr.sub(atr.mean()).div(atr.std())
    return z_atr

df['atr_zscore'] = compute_atr(df)

def compute_macd(close):
    macd = pandas_ta.macd(close=close, length=20).iloc[:, 0]
    return macd.sub(macd.mean()).div(macd.std())

df['macd'] = compute_macd(df['close'])
df['dollar_volume'] = (df['close'] * df['volume'])/1e6
last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume', 'open', 'high', 'low', 'volume']]

adjusted_volume = df['dollar_volume'].resample('ME').mean().to_frame('dollar_volume')
df_monthly = df[last_cols].resample('ME').last()

data = (pd.concat([adjusted_volume, df_monthly], axis=1)).dropna()
data['dollar_volume'] = data['dollar_volume'].rolling(5*12).mean().dropna()
data['dollar_vol_rank'] = (data['dollar_volume'].rank(ascending=False)).dropna()
data = data[data['dollar_vol_rank']<25].drop(['dollar_volume', 'dollar_vol_rank'], axis=1)

''
def calculate_returns(df):
    outlier_cutoff = 0.005
    lags = [1, 2, 3, 6, 9, 12]

    for lag in lags:
        df[f'return_{lag}m'] = (df['close']
                                .pct_change(lag)
                                .pipe(lambda x: x.clip(lower=x.quantile(outlier_cutoff),
                                                       upper=x.quantile(1-outlier_cutoff)))
                                .add(1)
                                .pow(1/lag)
                                .sub(1))
    return df

data = calculate_returns(data).dropna()
data

# web.DataReader('F-F_Research_data_5_factors_2x3',
#                'famafrench',
#                start='2010')



# print(df)
# print(type(df.index))       # Shows the index type
# print(df.index.nlevels) 
# print(df.index.names)
# print(df.columns)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


,close,garman_klass_vol,rsi,bb_low,bb_mid,bb_high,atr_zscore,macd,return_1m,return_2m,return_3m,return_6m,return_9m,return_12m
Date,,,,,,,,,,,,,,
2024-05-31,520.742249,0.000133,58.010101,6.229914,6.251467,6.273019,0.027699,0.680897,0.050580,0.004102,0.013546,0.025602,0.018904,0.020810
2024-06-30,539.114136,0.000079,65.477827,6.255094,6.281394,6.307694,-0.164489,1.135859,0.035280,0.042902,0.014389,0.023899,0.028375,0.018421
2024-07-31,545.642334,0.000054,54.500258,6.281033,6.306774,6.332516,0.677774,-0.199590,0.012109,0.023629,0.032535,0.023257,0.031997,0.016711
2024-08-31,558.391541,0.000069,59.902953,6.240058,6.297212,6.354366,0.897516,0.962489,0.023366,0.017722,0.023541,0.018531,0.024915,0.020061
2024-09-30,570.120361,0.000048,63.190769,6.285237,6.322899,6.360561,0.527820,1.216273,0.021005,0.022184,0.018815,0.016599,0.022202,0.025976
2024-10-31,565.032898,0.000022,47.389810,6.339766,6.356177,6.372588,0.284295,0.378371,-0.008923,0.005929,0.011708,0.022068,0.019393,0.027026
2024-11-30,598.727722,0.000013,64.484676,6.347083,6.378050,6.409017,0.246110,1.016357,0.059633,0.024782,0.023521,0.023531,0.020192,0.024566
2024-12-31,584.322266,0.000040,45.465246,6.368058,6.392563,6.417068,0.749386,-0.391778,-0.024060,0.016926,0.008235,0.013511,0.013804,0.018692
2025-01-31,600.015015,0.000074,53.929669,6.358035,6.388460,6.418886,0.890064,0.454353,0.026856,0.001074,0.020225,0.015958,0.021454,0.019601
